# Identifying Customer Segments

## Project

Datasets for this project were provided by the Bertlesmann Arvato Analytics. They concern the problem of a company performing mail-order sales in Germany. The company wants to target marketing campaigns towards the people that are most likely to purchase their products. The goal of the project is to identify customer segments with this in mind.

Project involves dealing with messiness and robustness that comes with real life data. There are two datasets, one about the company's customers and the other one about general german population. The information from both of them needs to be matched to perform the analysis. The final step is to use clustering algorithms to solve the company's problem.